In [1]:
%load_ext autoreload
%autoreload
import warnings; warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
# Python dependences
import os, time
import pandas as pd
import numpy as np   # Numpy - Python's numerical library
import matplotlib.pyplot as plt  # Matplotlib - Python's plotting library
from copy import deepcopy   # Python module for copying objects

# Input and Output Tools
# Catalogue and sources 
from openquake.hmtk.parsers.catalogue import CsvCatalogueParser   # Reads an earthquake catalogue from CSV
from openquake.hmtk.parsers.catalogue.csv_catalogue_parser import CsvCatalogueWriter  # Writes an earthquake catalogue to CSV
from openquake.hmtk.parsers.source_model.nrml04_parser import nrmlSourceModelParser  # Imports a source model from XML

# Completeness tools
from openquake.hmtk.seismicity.completeness.comp_stepp_1971 import Stepp1971

# Plotting tools
from openquake.hmtk.plotting.seismicity.completeness import plot_stepp_1972
from openquake.hmtk.plotting.seismicity.catalogue_plots import plot_magnitude_time_scatter
from openquake.hmtk.plotting.seismicity.catalogue_plots import plot_depth_histogram
from openquake.hmtk.plotting.seismicity.catalogue_plots import plot_magnitude_time_density
from openquake.hmtk.plotting.seismicity.max_magnitude.cumulative_moment import plot_cumulative_moment 
from openquake.hmtk.plotting.seismicity.occurrence.recurrence_plot import plot_recurrence_model
from openquake.hmtk.plotting.seismicity.catalogue_plots import (plot_observed_recurrence, 
                                                      get_completeness_adjusted_table,
                                                      _get_catalogue_bin_limits)

# Recurrence and Mmax
from openquake.hmtk.seismicity.occurrence.kijko_smit import KijkoSmit
from openquake.hmtk.seismicity.occurrence.weichert import Weichert
from openquake.hmtk.seismicity.max_magnitude.kijko_sellevol_bayes import KijkoSellevolBayes
from openquake.hmtk.seismicity.max_magnitude.kijko_sellevol_fixed_b import KijkoSellevolFixedb
from openquake.hmtk.seismicity.max_magnitude.kijko_nonparametric_gaussian import KijkoNonParametricGaussian
from openquake.hmtk.seismicity.max_magnitude.cumulative_moment_release import CumulativeMoment
from openquake.hmtk.plotting.seismicity.max_magnitude.cumulative_moment import plot_cumulative_moment

# Catalogue selector - we will see this in action
from openquake.hmtk.seismicity.selector import CatalogueSelector

from openquake.hazardlib.mfd import TruncatedGRMFD

In [3]:
Depth = "Shallow" # Select if for 'Shallow', 'MidCrust', or 'Deep'

In [8]:
output_dir = r'C:\Users\AMH-L91\Documents\Projects\openquake-notebooks\output_data'
catalogue_file = os.path.join(output_dir, "Overall_Seismicity_Dec_2024.09.17.csv")

parser = CsvCatalogueParser(catalogue_file)
catalogue = parser.read_file(start_year=1619)

if Depth == 'Shallow':
    catalogue.purge_catalogue(catalogue.data['depth'] <= 50)
    
elif Depth == 'MidCrust':
    catalogue.purge_catalogue(catalogue.data['depth'] > 50 and 
                              catalogue.data['depth'] <= 100)
elif Depth == 'Deep':
    catalogue.purge_catalogue(catalogue.data['depth'] > 100)

print(f'Number of mainshocks from {Depth} sources: {catalogue.get_number_events()}')

Number of mainshocks from Shallow sources: 11282


# Load in a Source Model File

In [5]:
dir_path = r'C:\Users\AMH-L91\Documents\Projects\openquake-notebooks\input_data'
source_model_file = os.path.join(dir_path, "AreaSource_" + Depth + ".xml")

parser = nrmlSourceModelParser(source_model_file)

# Parse the seismic sources and save them into a variable called "source_model"
source_model = parser.read_file("AreaSource_" + Depth) # You need to supply a name for the source model

### A Source Model Workflow - Area Source Example

Each of the area sources has:

1. Defined name, ID, tectonic region type

2. Geometry (polygon)

3. Magnitude Scaling Relation (Wells & Coppersmith, 1994)

4. Rupture Aspect Ratio (1.5)

5. Nodal Plane Distribution (varies)


But they are missing the magnitude frequency distribution and the hypocentral depth distribution!

Using the observed catalogue - define the MFD as a Truncated Gutenberg-Richer model for each source

Assume $M_{MIN} = 4.8$

### Completeness of Sub-Catalog

In [ ]:
# Set up the configuration parameters
comp_config = {'magnitude_bin': 0.5, 'time_bin': 5.0, 'increment_lock': True}

# Calling the method by Stepp (1971)
completeness_algorithm = Stepp1971()

# Use the catalogue and completeness configuration
completeness_table = completeness_algorithm.completeness(catalogue, comp_config)
print('Completeness: ok')

# Print the completeness table
print('\n')
print('Completeness table using Stepp method (1971)')
print(completeness_table)
print('\n')

# Setting configuration for the completeness plot
completeness_parameters = completeness_algorithm

output_file = os.path.join(output_dir, Depth + "_Completeness_Overall_Plot.png")
if os.path.exists(output_file):
    os.remove(output_file)

plot_stepp_1972.create_stepp_plot(completeness_parameters, figure_size=(8, 6), 
                                  filename=output_file, filetype='png', dpi=300, ax=None)

In [8]:
# Trim catalogue for plotting purposes only
cat_dec_plot = deepcopy(catalogue)
cat_years = cat_dec_plot.data['year']
cat_mon = cat_dec_plot.data['month']
cat_day = cat_dec_plot.data['day']
cat_hr = cat_dec_plot.data['hour']
cat_min = cat_dec_plot.data['minute']
cat_sec = cat_dec_plot.data['second']
cat_mags = cat_dec_plot.data['magnitude']

cat_recent_years = cat_years >= 1900
cat_dec_plot.data['year'] = cat_years[cat_recent_years]
cat_dec_plot.data['month'] = cat_mon[cat_recent_years]
cat_dec_plot.data['day'] = cat_day[cat_recent_years]
cat_dec_plot.data['hour'] = cat_hr[cat_recent_years]
cat_dec_plot.data['minute'] = cat_min[cat_recent_years]
cat_dec_plot.data['second'] = cat_sec[cat_recent_years]
cat_dec_plot.data['magnitude'] = cat_mags[cat_recent_years]

In [ ]:
comp_tables = {
    'Shallow': np.array([[1980., 4.8],
                         [1964., 5.0],
                         [1955., 5.5],
                         [1945., 6.0],
                         [1930., 6.5],
                         [1920., 7.0],
                         [1863., 7.5],
                         [1619., 8.2]]),
    'MidCrust': np.array([[1976., 4.8],
                         [1963., 5.0],
                         [1957., 5.9],
                         [1949., 6.6],
                         [1903., 7.0],
                         [1884., 7.2],
                         [1852., 7.6]]),
    'Deep': np.array([[1986., 4.8],
                     [1963., 5.5],
                     [1871., 7.0]])
}

completeness_table_a = comp_tables[Depth]

plot_magnitude_time_density(cat_dec_plot, 0.1, 1.0,
                            completeness=completeness_table_a)

min_mag = 4.8

selector1 = CatalogueSelector(catalogue, create_copy=True)

In the following workflow we will, for each source:

1. Select the earthquakes within the source

2. Plot them on a map

3. Use them to calculate a- and b-value from the Weichert (1980) method

4. Estimate Mmax using the cumulative moment method (plus a small offset)

5. View the hypocentral depth distribution - and use the density to define the hypocentral depth distribution for the source

In [ ]:
# Add to dictionary the source models where hypodepth dists are to be added/updated
IdDict = {
    'Shallow': ['1','2','3','4','5','6C','8','9'],
    'MidCrust': ['11AB','11C','12','13A'],
    'Deep': ['14BC','15A']
}
IdList = IdDict[Depth]

df_gr = pd.DataFrame(columns=['Zone', 'Source', 'Type', 'a', 'sigma_a', 'b', 'sigma_b'])

for source in source_model.sources:

    print ('----------------------------------------------------------------------------------------------------')
    # Select the earthquakes within the source
    source.select_catalogue(selector1)
    print("Source ID: %s  Source Name: %s   Tectonic Region: %s   Number of Events: %g" % (source.id, 
                                                                                           source.name, 
                                                                                           source.trt,
                                                                                        source.catalogue.get_number_events()))

    # Get the a- and b-value using Weichert (1980)
    occurrence = Weichert()
    recurrence_config = {"magnitude_interval": 0.1}
    bval, sigma_b, aval, sigma_a = occurrence.calculate(source.catalogue,
                                                        recurrence_config,
                                                        completeness_table_a)
    print("a = %.3f (+/- %.3f),  b = %.3f (+/-%.3f)" % (aval, sigma_a, bval, sigma_b))
    df_gr = df_gr.append({'Zone': source.id, 'Source': source.name, 'Type': source.trt, 
                          'a': aval, 'sigma_a': sigma_a, 'b': bval, 'sigma_b': sigma_b},
                        ignore_index=True)
    
    # Estimate the Maximum Magnitude - using the Kijko & Smit Nonparametric Gaussian
    mmax_config = {"input_mmin": 4.8, 
                   "input_mmax": None, 
                   "input_uncertainty": None,
                   "b-value": bval,
                   "sigma-b": sigma_b,
                   "number_earthquakes": 5,
                  }
    
    #mmax_config = {
    #              "number_bootstraps": 1000
    #              }
    mmax_calculator = KijkoNonParametricGaussian()

    mmax, sigma_mmax = mmax_calculator.get_mmax(source.catalogue, mmax_config)
    print("Mmax (Observed) = %.2f Mmax (Inferred) = %.3f +/- %.3f" % (
        np.max(source.catalogue.data["magnitude"]), mmax + 0.2, sigma_mmax))
    
    # Compare the model against data
    source.mfd = TruncatedGRMFD(min_mag, mmax + 0.2, 0.1, aval, bval)
    
    plot_magnitude_time_density(source.catalogue, 0.1, 1.0)

    if source.id in IdList:

        # Show the depth histogram
        if Depth == 'Deep': depth_interval = 100.
        else: depth_interval = 25.  # in km
        # 50-70 deep; 30 otherwise
        
        plot_depth_histogram(source.catalogue, depth_interval, normalisation=True)
        plot_recurrence_model(source.mfd, source.catalogue, completeness_table_a, 0.1)
    
        # Add the probability mass function of the hypocentre depths to the source
        depth_bins = np.arange(source.upper_depth,
                               source.lower_depth + depth_interval,
                               depth_interval)
        source.hypo_depth_dist = source.catalogue.get_depth_pmf(depth_bins)
        # Wait for the plotting to catch up!
        
        print("Upper Seismogenic Depth: %.2f, Lower Seismogenic Depth: %.2f" %(source.upper_depth, source.lower_depth))

    time.sleep(1.0)

In [11]:
source_model.serialise_to_nrml(os.path.join(output_dir, "AreaSource_" + Depth + ".xml"))

Export to csv for copying

In [12]:
df_gr.to_csv(os.path.join(output_dir, 'Area_Source_GR-Parameters_' + Depth + '.csv'), index=False)